In [1]:
from fastai.vision.all import *

In [2]:
path = Path("/notebooks/tomatodiagnosis/test")

In [3]:
def train(arch, item, batch, epochs=5):
    dls = ImageDataLoaders.from_folder(path, seed=42, item_tfms=item, valid_pct=0.2, batch_tfms=batch)
    learn = vision_learner(dls, arch, metrics=error_rate).to_fp16()
    learn.fine_tune(epochs, 0.01)
    return learn

In [6]:
learn = train('resnet26d', item=Resize(224), batch=aug_transforms(size=128, min_scale=0.75))

epoch,train_loss,valid_loss,error_rate,time
0,2.161346,1.302750,0.417676,00:18


epoch,train_loss,valid_loss,error_rate,time
0,1.556464,1.099046,0.353524,00:22
1,1.238174,0.880741,0.276982,00:22
2,1.086626,0.774862,0.234031,00:22
3,1.021572,0.739330,0.227974,00:22
4,0.997933,0.731093,0.226046,00:22


In [7]:
learn = train('convnext_small_in22k', item=Resize(224), batch=aug_transforms(size=128, min_scale=0.75))

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_small_22k_224.pth" to /root/.cache/torch/hub/checkpoints/convnext_small_22k_224.pth


epoch,train_loss,valid_loss,error_rate,time
0,1.659661,0.801909,0.244218,00:31


epoch,train_loss,valid_loss,error_rate,time
0,1.019441,0.671423,0.209251,00:43
1,0.804069,0.504386,0.165749,00:43
2,0.644734,0.433796,0.145650,00:43
3,0.597413,0.393527,0.132434,00:43
4,0.572031,0.379120,0.121971,00:44


In [9]:
learn = train('vit_small_patch16_224', item=Resize(240), batch=aug_transforms(size=224, min_scale=0.75))

epoch,train_loss,valid_loss,error_rate,time
0,1.620360,0.777772,0.242291,00:59


epoch,train_loss,valid_loss,error_rate,time
0,1.032339,0.665177,0.210903,01:07
1,0.825204,0.520719,0.161894,01:07
2,0.691887,0.456924,0.141795,01:07
3,0.674393,0.420244,0.131608,01:07
4,0.635378,0.417575,0.126927,01:07


In [4]:
learn = train('swin_s3_tiny_224', item=Resize(240), batch=aug_transforms(size=224, min_scale=0.75))

/usr/local/lib/python3.9/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/s3_t-1d53f6a8.pth" to /root/.cache/torch/hub/checkpoints/s3_t-1d53f6a8.pth


epoch,train_loss,valid_loss,error_rate,time
0,0.437547,0.244028,0.075991,02:34


epoch,train_loss,valid_loss,error_rate,time
0,0.139527,0.054785,0.016244,03:21
1,0.104683,0.060351,0.015694,03:21
2,0.057571,0.032782,0.009912,03:22
3,0.023636,0.011396,0.004130,03:21
4,0.010157,0.012194,0.003855,03:21


In [ ]:
learn = train('convnext_small_in22k', item=Resize(192), batch=aug_transforms(size=128, min_scale=0.75))

In [ ]:
learn = train('convnext_small_in22k', item=Resize(192, method='squish'), batch=aug_transforms(size=128, min_scale=0.75))

In [ ]:
dls = ImageDataLoaders.from_folder(path, valid_pct=0.2, seed=42,
    item_tfms=Resize(192, method=ResizeMethod.Pad, pad_mode=PadMode.Zeros))
dls.show_batch(max_n=3)

In [ ]:
learn = train('convnext_small_in22k', epochs=12,
              item=Resize(256, method='squish'), batch=aug_transforms(size=128, min_scale=0.75))

In [ ]:
learn.path = Path('.')
learn.export('01_tomato.pkl')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(2, nrows=1)

In [ ]:
learn.dls.vocab

In [ ]:
path = Path("/notebooks/tomatodiagnosis/")

In [2]:
pwd

'/notebooks/tomatodiagnosis'

In [2]:
learn_inf = load_learner('01_tomato.pkl')

In [4]:
#learn_inf.model

In [5]:
dummy_input = torch.randn(1, 3, 224, 224)

In [6]:
torch.onnx.export(learn_inf.model, dummy_input, "my_model.onnx")

In [2]:
import onnx

model = onnx.load("my_model.onnx")

In [3]:
onnx.checker.check_model(model)

In [4]:
print(onnx.helper.printable_graph(model.graph))

graph torch_jit (
  %input.1[FLOAT, 1x3x224x224]
) initializers (
  %0.model.stem.0.weight[FLOAT, 96x3x4x4]
  %0.model.stem.0.bias[FLOAT, 96]
  %0.model.stem.1.weight[FLOAT, 96]
  %0.model.stem.1.bias[FLOAT, 96]
  %0.model.stages.0.blocks.0.conv_dw.weight[FLOAT, 96x1x7x7]
  %0.model.stages.0.blocks.0.conv_dw.bias[FLOAT, 96]
  %0.model.stages.0.blocks.0.norm.weight[FLOAT, 96]
  %0.model.stages.0.blocks.0.norm.bias[FLOAT, 96]
  %0.model.stages.0.blocks.0.mlp.fc1.bias[FLOAT, 384]
  %0.model.stages.0.blocks.0.mlp.fc2.bias[FLOAT, 96]
  %0.model.stages.0.blocks.1.conv_dw.weight[FLOAT, 96x1x7x7]
  %0.model.stages.0.blocks.1.conv_dw.bias[FLOAT, 96]
  %0.model.stages.0.blocks.1.norm.weight[FLOAT, 96]
  %0.model.stages.0.blocks.1.norm.bias[FLOAT, 96]
  %0.model.stages.0.blocks.1.mlp.fc1.bias[FLOAT, 384]
  %0.model.stages.0.blocks.1.mlp.fc2.bias[FLOAT, 96]
  %0.model.stages.0.blocks.2.conv_dw.weight[FLOAT, 96x1x7x7]
  %0.model.stages.0.blocks.2.conv_dw.bias[FLOAT, 96]
  %0.model.stages.0.blocks.2

In [ ]:
img = '/notebooks/tomatodiagnosis/septoria-leaf-spot2.jpg'
img = PILImage.create(img)
img

learn_inf.predict(img)

import gradio as gr
labels = learn_inf.dls.vocab
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn_inf.predict(img)
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

In [ ]:
gr.Interface(fn=predict, inputs=gr.inputs.Image(shape=(480, 360)), outputs=gr.outputs.Label(num_top_classes=1)).launch(share=True)